In [1]:
import pandas as pd
# df = pd.read_csv("data.csv")
df = pd.read_csv("data.csv.bak")
print(df.columns)
df = df[["title", "points", "comments", "viewed"]]
df["viewed"] = df["viewed"].apply((lambda x: 1 if x==1 else 0))
df.loc[df["viewed"].isna(), "viewed"] = 0
df.loc[[1,3,10,12,18,19,21],"viewed"] = 1
df

Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'rank', 'title', 'url', 'points',
       'comments', 'viewed', 'liked'],
      dtype='object')


,title,points,comments,viewed
0,The greatest risk of AI is from the people who...,194,77,0
1,Launch HN: Onu (YC W23) – Turn scripts into in...,69,41,1
2,OpenAI's plans according to sama (humanloop.com),63,25,0
3,Rarbg Is No More (archive.org),800,452,1
4,Slide to Unlock (uwaterloo.ca),135,28,0
5,Zulip 7.0: Threaded open-source team chat (zul...,46,11,0
6,Improving Mathematical Reasoning with Process ...,50,7,0
7,"Tiny, medical robots could one day travel thro...",190,79,0
8,macOS Like Fonts on Manjaro/Arch Linux (aswinm...,9,3,0
9,Rise: Accelerate the Development of Open Sourc...,35,6,0


In [2]:
import numpy as np
import torch
import transformers as ppb

# pretrained_weights_name = "distilbert-base-uncased"
pretrained_weights_name = "bert-base-uncased"
tokenizer = ppb.DistilBertTokenizer.from_pretrained(pretrained_weights_name)
model = ppb.DistilBertModel.from_pretrained(pretrained_weights_name)

tokenized = df["title"].apply((lambda s: tokenizer.encode(s, add_special_tokens=True)))

max_len = max([len(row) for row in tokenized])
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

print(padded.shape)
padded

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.
You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing DistilBertModel: ['bert.encoder.layer.9.output.dense.bias', 'bert.encoder.layer.11.attention.output.LayerNorm.bias', 'bert.encoder.layer.10.attention.output.LayerNorm.weight', 'bert.encoder.layer.7.attention.self.key.bias', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.8.attention.self.query.bias', 'bert.encoder.layer.1.attention.self.key.bias', 'bert.encoder.layer.11.output.LayerNorm.weight', 'bert.encoder.layer.3.attention.self.key.bias

(29, 33)


array([[  101,  1996,  4602,  3891,  1997,  9932,  2003,  2013,  1996,
         2111,  2040,  2491,  2009,  1010,  2025,  1996,  6627,  2993,
         1006,  9932,  2015,  2532,  3489, 10448,  2140,  1012,  4942,
         9153,  3600,  1012,  4012,  1007,   102],
       [  101,  4888,  1044,  2078,  1024,  2006,  2226,  1006,  1061,
         2278,  1059, 21926,  1007,  1516,  2735, 14546,  2046,  4722,
         5906,  1999,  2781,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0],
       [  101,  2330,  4886,  1005,  1055,  3488,  2429,  2000,  3520,
         2050,  1006,  2529,  4135,  7361,  1012,  4012,  1007,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0],
       [  101, 10958, 15185,  2290,  2003,  2053,  2062,  1006,  8756,
         1012,  8917,  1007,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,   

In [3]:
encoded = torch.tensor(padded)
attention_mask = np.where(padded != 0, 1, 0)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(encoded, attention_mask=attention_mask)

# last_hidden_states[0].shape



features = last_hidden_states[0][:,0,:].numpy()
print(features.shape)
labels = df["viewed"].to_list()

# add features
new_features = ["points","comments"]
features = np.append(features, df[new_features].to_numpy(), axis=1)
print(features.shape)

features, labels

(29, 768)
(29, 770)


(array([[ 4.47852731e-01,  1.61816251e+00,  4.20298845e-01, ...,
          2.13210061e-01,  1.94000000e+02,  7.70000000e+01],
        [ 2.78446943e-01,  1.97484410e+00,  2.18175188e-01, ...,
          5.90770841e-01,  6.90000000e+01,  4.10000000e+01],
        [-6.53122887e-02,  1.70917666e+00,  9.57809836e-02, ...,
          1.54796004e-01,  6.30000000e+01,  2.50000000e+01],
        ...,
        [ 6.22555375e-01,  1.70079875e+00,  2.26779312e-01, ...,
          3.26630741e-01,  1.32000000e+02,  1.25000000e+02],
        [ 5.93542278e-01,  1.46398520e+00,  3.04184079e-01, ...,
          2.16497585e-01,  2.20000000e+01,  4.00000000e+00],
        [ 5.43479562e-01,  2.00709319e+00,  3.45604748e-01, ...,
          4.21110868e-01,  7.84000000e+02,  6.33000000e+02]]),
 [0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0])

In [4]:
features = last_hidden_states[0][:,0,:].numpy()
labels = df["viewed"]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.4, random_state=42)

from sklearn.linear_model import LogisticRegression, LinearRegression
clf = LogisticRegression(random_state=42)
lin_clf = LinearRegression()
clf.fit(x_train, y_train)
print(clf.score(x_test, y_test))
lin_clf.fit(x_train, y_train)
print(lin_clf.score(x_test, y_test))

0.9166666666666666
0.609482170416332


In [5]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
clf = DummyClassifier()
scores = cross_val_score(clf, x_train, y_train)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.700 (+/- 0.08)
